# 5주차 — 시그널 융합과 이상징후 탐지


## 학습 목표
- 텍스트 시그널 + 기상 위험지표 결합, 이동창 z-score 기반 이상 탐지

In [ ]:

import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from utils import zscore, risk_index_rulebased

weather = pd.read_csv(str(Path("..")/"data"/"sample_weather_2022.csv"))
weather['date'] = pd.to_datetime(weather['date'])
weather['risk'] = weather.apply(risk_index_rulebased, axis=1)
weather['risk_7d'] = weather['risk'].rolling(7, min_periods=1).sum()

news = pd.read_csv(str(Path("..")/"data"/"news_samples_labeled.csv"))
# 주 단위 시그널(간단 집계)
news_daily = news.groupby('date', as_index=False)['label_signal'].mean().rename(columns={'label_signal':'news_signal'})
news_daily['date'] = pd.to_datetime(news_daily['date'])

df = pd.merge(weather[['date','risk_7d']], news_daily, on='date', how='left').fillna(0.0)
df['fusion'] = zscore(df['risk_7d']) + zscore(df['news_signal'])

plt.figure(figsize=(10,4))
plt.plot(df['date'], df['fusion'])
plt.title("융합 조기경보 지수 (표준화 합)")
plt.xlabel("날짜"); plt.ylabel("지수")
plt.tight_layout(); plt.show()
